# Notebook to calculate statistics about circuits

In [1]:
import warnings
import json
import os
import sys
import glob
import numpy
import pickle
import pennylane as qml

from math import ceil
from pathlib import Path
from jax import numpy as np
from sympy import default_sort_key
from utils import *

this_folder = os.path.abspath(os.getcwd())

In [2]:
# Select workload
#workload = "execution_time"
workload = "cardinality"

# Select workload size
#workload_size = "small"
#workload_size = "medium"
#workload_size = "large"
workload_size = "main"

classification = 2
layers = 1
single_qubit_params = 3
n_wire_count = 1

# Access the selected circuits
path_name = this_folder + "//simplified-JOB-diagrams//" + workload + "//" + workload_size + "//circuits//" + str(classification) + "//" + str(layers) + "_layer//" + str(single_qubit_params) + "_single_qubit_params//" + str(n_wire_count) + "_n_wire_count//"

In [3]:
training_circuits_paths = glob.glob(path_name + "training//[0-9]*.p")
validation_circuits_paths = glob.glob(path_name + "validation//[0-9]*.p")
test_circuits_paths = glob.glob(path_name + "test//[0-9]*.p")

In [4]:
training_circuits = read_diagrams(training_circuits_paths)
validation_circuits = read_diagrams(validation_circuits_paths)
test_circuits = read_diagrams(test_circuits_paths)

In [5]:
n_qubits = 20
dev = qml.device("default.qubit", wires=n_qubits, shots=1)

qml_training_circuits, train_symbols = transform_into_pennylane_circuits(training_circuits.values(), n_qubits, dev)
qml_test_circuits, test_symbols = transform_into_pennylane_circuits(test_circuits.values(), n_qubits, dev)
qml_validation_circuits, val_symbols = transform_into_pennylane_circuits(validation_circuits.values(), n_qubits, dev)
all_circuits = qml_training_circuits + qml_test_circuits + qml_validation_circuits

In [6]:
numpy.random.seed(0)
rng = numpy.random.default_rng(0)
result = {}

for circuit in all_circuits:
    init_params = np.array(rng.random(len(train_symbols)))
    specs_func = qml.specs(circuit)
    executed_circuit = specs_func(init_params)
    gate_types = executed_circuit["gate_types"]
    for elem in gate_types:
        if elem not in result:
            result[elem] = gate_types[elem]
        else:
            result[elem] = result[elem] + gate_types[elem] 

# Classification task
# Number of classes
# Number of training circuits
# Number of validation circuits
# Number of test circuits
# Total number of circuits
# Total number of variables
# Single qubit gates in total:
    # Hadamard
    # Rx
    # Rz
# Two-qubit gates
    # Ctrl-Rx
    # Ctrl-Rz

print(workload, 2**classification, len(qml_training_circuits), len(qml_validation_circuits), len(qml_test_circuits), len(qml_training_circuits)+len(qml_validation_circuits)+len(qml_test_circuits), len(train_symbols), result)

cardinality 4 448 113 112 673 279 {'Hadamard': 10774, 'CRZ': 6060, 'RX': 30982, 'RZ': 5387}
